# Administrador de Base de Datos - Programas de Doctorado

Este notebook permite administrar la base de datos de programas de doctorado y visualizar la información.

## Funcionalidades:
1. Conexión a MongoDB
2. Backup y restauración de datos
3. Enriquecimiento de datos mediante OpenAI
4. Visualización de datos (mapas, gráficos radiales)
5. Consultas y filtros personalizados

## Configuración Inicial

Primero instalamos las dependencias necesarias.

In [ ]:
# Instalar dependencias necesarias
!pip install pymongo pandas matplotlib seaborn plotly folium ipywidgets openai python-dotenv

## Importar librerías y configurar el entorno

In [ ]:
import os
import json
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import folium
from folium.plugins import MarkerCluster
from pymongo import MongoClient
from bson.objectid import ObjectId
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import openai
from dotenv import load_dotenv

# Cargar variables de entorno desde .env
load_dotenv()

# Configurar OpenAI
openai.api_key = os.getenv('OPENAI_API_KEY')

# Configuración de visualización
plt.style.use('ggplot')
sns.set(style="whitegrid")

# Definir estilos de colores para los estados
status_colors = {
    'pendiente': '#778ca3',
    'considerando': '#4b6584',
    'interesado': '#3c40c6',
    'aplicando': '#0abde3',
    'descartado': '#ee5253'
}

# Configurar mensaje de éxito
def success_msg(msg):
    display(HTML(f'<div style="background-color: #d4edda; color: #155724; padding: 10px; border-radius: 5px; margin: 10px 0;">{msg}</div>'))

# Configurar mensaje de error
def error_msg(msg):
    display(HTML(f'<div style="background-color: #f8d7da; color: #721c24; padding: 10px; border-radius: 5px; margin: 10px 0;">{msg}</div>'))

# Configurar mensaje de información
def info_msg(msg):
    display(HTML(f'<div style="background-color: #cce5ff; color: #004085; padding: 10px; border-radius: 5px; margin: 10px 0;">{msg}</div>'))

## Conexión a MongoDB

In [ ]:
# Función para conectar a MongoDB
def connect_mongodb():
    try:
        uri = os.getenv('MONGODB_URI', 'mongodb://localhost:27017/doctorados')
        client = MongoClient(uri)
        db = client.get_database()
        success_msg(f"Conexión exitosa a MongoDB: {db.name}")
        return client, db
    except Exception as e:
        error_msg(f"Error al conectar a MongoDB: {str(e)}")
        return None, None

# Conectar a MongoDB
mongo_client, db = connect_mongodb()

## Funciones de Backup y Restauración

In [ ]:
# Función para hacer backup de la colección de programas
def backup_collection(source_collection='programas', backup_prefix='backup'):
    try:
        # Generar nombre para la colección de backup con fecha y hora
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        backup_collection = f"{backup_prefix}_{source_collection}_{timestamp}"
        
        # Obtener todos los documentos de la colección original
        docs = list(db[source_collection].find({}))
        
        # Si no hay documentos, mostrar un mensaje y salir
        if not docs:
            info_msg(f"La colección '{source_collection}' está vacía. No se creó backup.")
            return None
        
        # Crear la colección de backup e insertar los documentos
        db.create_collection(backup_collection)
        db[backup_collection].insert_many(docs)
        
        # Mostrar información del backup
        success_msg(f"Backup exitoso: '{source_collection}' → '{backup_collection}' ({len(docs)} documentos)")
        return backup_collection
    except Exception as e:
        error_msg(f"Error al crear backup: {str(e)}")
        return None

# Función para restaurar desde un backup
def restore_from_backup(backup_collection, target_collection='programas', overwrite=False):
    try:
        # Verificar si la colección de backup existe
        if backup_collection not in db.list_collection_names():
            error_msg(f"La colección de backup '{backup_collection}' no existe.")
            return False
        
        # Obtener documentos del backup
        docs = list(db[backup_collection].find({}))
        if not docs:
            info_msg(f"La colección de backup '{backup_collection}' está vacía.")
            return False
        
        # Si la colección destino existe y overwrite=True, eliminarla
        if target_collection in db.list_collection_names():
            if overwrite:
                db[target_collection].drop()
                info_msg(f"Colección '{target_collection}' eliminada para restauración.")
            else:
                error_msg(f"La colección '{target_collection}' ya existe. Use overwrite=True para sobrescribir.")
                return False
        
        # Crear la colección destino si no existe
        if target_collection not in db.list_collection_names():
            db.create_collection(target_collection)
        
        # Insertar documentos en la colección destino
        db[target_collection].insert_many(docs)
        success_msg(f"Restauración exitosa: '{backup_collection}' → '{target_collection}' ({len(docs)} documentos)")
        return True
    except Exception as e:
        error_msg(f"Error al restaurar desde backup: {str(e)}")
        return False

# Función para listar todos los backups disponibles
def list_backups(prefix='backup'):
    collections = db.list_collection_names()
    backups = [coll for coll in collections if coll.startswith(prefix)]
    
    if not backups:
        info_msg("No hay backups disponibles.")
        return []
    
    # Ordenar backups por fecha (del más reciente al más antiguo)
    backups.sort(reverse=True)
    
    # Mostrar lista de backups
    print("Backups disponibles:")
    for i, backup in enumerate(backups, 1):
        count = db[backup].count_documents({})
        print(f"{i}. {backup} ({count} documentos)")
    
    return backups

### Crear un backup de la base de datos

In [ ]:
# Crear un backup de la colección programas
backup_name = backup_collection()

### Listar backups disponibles

In [ ]:
# Listar todos los backups disponibles
available_backups = list_backups()

### Restaurar desde un backup

⚠️ **Precaución**: Esta operación sobrescribirá la colección actual si se establece `overwrite=True`.

In [ ]:
# Ejemplo: Restaurar desde el backup más reciente
if available_backups:
    # Crear un widget dropdown para seleccionar el backup a restaurar
    backup_dropdown = widgets.Dropdown(
        options=available_backups,
        description='Backup:',
        style={'description_width': 'initial'}
    )
    
    overwrite_checkbox = widgets.Checkbox(
        value=False,
        description='Sobrescribir colección existente',
        style={'description_width': 'initial'}
    )
    
    restore_button = widgets.Button(
        description='Restaurar',
        button_style='danger',
        tooltip='Restaurar desde backup seleccionado'
    )
    
    output = widgets.Output()
    
    def on_restore_button_click(b):
        with output:
            clear_output()
            if not backup_dropdown.value:
                error_msg("Seleccione un backup para restaurar.")
                return
            
            print(f"Restaurando desde {backup_dropdown.value}...")
            restore_from_backup(backup_dropdown.value, overwrite=overwrite_checkbox.value)
    
    restore_button.on_click(on_restore_button_click)
    
    display(widgets.VBox([
        widgets.HBox([backup_dropdown, overwrite_checkbox]),
        restore_button,
        output
    ]))
else:
    print("No hay backups disponibles para restaurar.")

## Funciones de Enriquecimiento de Datos con OpenAI

In [ ]:
# Función para obtener coordenadas geográficas
def get_coordinates(ciudad, country="España"):
    import requests
    try:
        response = requests.get("https://nominatim.openstreetmap.org/search", 
                               params={
                                   "city": ciudad,
                                   "country": country,
                                   "format": "json",
                                   "limit": 1
                               },
                               headers={
                                   "User-Agent": "GraduateProgramsEvaluator/1.0"
                               })
        if response.status_code == 200 and response.json():
            data = response.json()[0]
            return {
                "lat": float(data["lat"]),
                "lon": float(data["lon"])
            }
        return None
    except Exception as e:
        print(f"Error obteniendo coordenadas para {ciudad}: {str(e)}")
        return None

# Función para generar resumen mediante OpenAI
def generate_summary(text):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "Eres un asistente académico especializado en resumir líneas de investigación científica de manera concisa y profesional."},
                {"role": "user", "content": f"Resume las siguientes líneas de investigación en un párrafo breve, destacando los aspectos más importantes y potenciales aplicaciones: {text}"}
            ],
            max_tokens=200,
            temperature=0.7
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error generando resumen: {str(e)}")
        return "No se pudo generar un resumen."

# Función para generar estadísticas mediante OpenAI
def generate_stats(universidad, programas):
    try:
        programas_texto = "\n\n".join([f"{p['nombre']}: {'. '.join(p['lineas_investigacion'])}" for p in programas])
        
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "Eres un analista de datos académicos que evalúa programas de doctorado y genera estadísticas cualitativas."},
                {"role": "user", "content": f"""Basándote en la siguiente información de programas de doctorado de {universidad}, genera 5 métricas numéricas en escala del 1 al 10 para evaluar: 
                1. Innovación: cuán innovadores son los temas de investigación
                2. Interdisciplinariedad: nivel de colaboración entre disciplinas
                3. Impacto potencial: posible impacto en la sociedad/industria
                4. Competitividad internacional: posicionamiento internacional
                5. Aplicabilidad: orientación práctica vs. teórica
                
                Programas y líneas de investigación:
                {programas_texto}
                
                Responde SOLO con un objeto JSON con este formato exacto:
                {{"innovacion": N, "interdisciplinariedad": N, "impacto": N, "internacional": N, "aplicabilidad": N}}
                donde N es un número del 1 al 10."""}
            ],
            max_tokens=150,
            temperature=0.7
        )
        
        content = response.choices[0].message.content.strip()
        try:
            return json.loads(content)
        except json.JSONDecodeError:
            print(f"Error al decodificar JSON de OpenAI: {content}")
            return {
                "innovacion": 5,
                "interdisciplinariedad": 5,
                "impacto": 5,
                "internacional": 5,
                "aplicabilidad": 5
            }
    except Exception as e:
        print(f"Error generando estadísticas: {str(e)}")
        return {
            "innovacion": 5,
            "interdisciplinariedad": 5,
            "impacto": 5,
            "internacional": 5,
            "aplicabilidad": 5
        }

# Función para obtener métricas de ciudad mediante OpenAI
def get_city_metrics(ciudad):
    metrics = {}
    
    # Obtener costo de vida
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "Eres un asistente especializado en economía y datos de ciudades españolas."},
                {"role": "user", "content": f"¿Cuál es el costo de vida aproximado en {ciudad} (sin incluir alquiler) en relación a Ciudad de México? Devuélvelo como un índice numérico 0–100, y un breve comentario."}
            ],
            max_tokens=100,
            temperature=0.5
        )
        costo_vida_text = response.choices[0].message.content.strip()
        # Extraer número del texto
        import re
        costo_vida_match = re.search(r'\b([0-9]{1,3})\b', costo_vida_text)
        metrics['costo_vida'] = int(costo_vida_match.group(0)) if costo_vida_match else 70
        metrics['costo_vida_comentario'] = costo_vida_text
    except Exception as e:
        print(f"Error obteniendo costo de vida para {ciudad}: {str(e)}")
        metrics['costo_vida'] = 70
        metrics['costo_vida_comentario'] = f"Error al obtener datos para {ciudad}"
    
    # Obtener calidad del servicio médico
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "Eres un asistente especializado en sistemas sanitarios españoles."},
                {"role": "user", "content": f"En una escala de 0 a 10, ¿qué puntuación le das a la calidad sanitaria en {ciudad}? Proporciónanos solo el número y, opcionalmente, dos frases de justificación."}
            ],
            max_tokens=100,
            temperature=0.5
        )
        medico_text = response.choices[0].message.content.strip()
        # Extraer número del texto
        import re
        medico_match = re.search(r'\b([0-9]|10)\b', medico_text)
        metrics['calidad_servicio_medico'] = int(medico_match.group(0)) if medico_match else 8
        metrics['calidad_servicio_medico_comentario'] = medico_text
    except Exception as e:
        print(f"Error obteniendo calidad de servicio médico para {ciudad}: {str(e)}")
        metrics['calidad_servicio_medico'] = 8
        metrics['calidad_servicio_medico_comentario'] = f"Error al obtener datos para {ciudad}"
    
    # Obtener calidad del transporte público
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "Eres un asistente especializado en infraestructura de transporte urbano."},
                {"role": "user", "content": f"En una escala de 0 a 10, ¿cómo calificarías la calidad de transporte público en {ciudad}? Responde con un número y una breve justificación."}
            ],
            max_tokens=100,
            temperature=0.5
        )
        transporte_text = response.choices[0].message.content.strip()
        # Extraer número del texto
        import re
        transporte_match = re.search(r'\b([0-9]|10)\b', transporte_text)
        metrics['calidad_transporte'] = int(transporte_match.group(0)) if transporte_match else 7
        metrics['calidad_transporte_comentario'] = transporte_text
    except Exception as e:
        print(f"Error obteniendo calidad de transporte para {ciudad}: {str(e)}")
        metrics['calidad_transporte'] = 7
        metrics['calidad_transporte_comentario'] = f"Error al obtener datos para {ciudad}"
    
    # Obtener calidad del aire
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "Eres un asistente especializado en calidad medioambiental urbana."},
                {"role": "user", "content": f"En una escala de 0 a 10, ¿cómo calificarías la calidad del aire en {ciudad}? Solo el número y, opcionalmente, una frase justificando."}
            ],
            max_tokens=100,
            temperature=0.5
        )
        aire_text = response.choices[0].message.content.strip()
        # Extraer número del texto
        import re
        aire_match = re.search(r'\b([0-9]|10)\b', aire_text)
        metrics['calidad_aire'] = int(aire_match.group(0)) if aire_match else 7
        metrics['calidad_aire_comentario'] = aire_text
    except Exception as e:
        print(f"Error obteniendo calidad del aire para {ciudad}: {str(e)}")
        metrics['calidad_aire'] = 7
        metrics['calidad_aire_comentario'] = f"Error al obtener datos para {ciudad}"
    
    return metrics

# Función principal para enriquecer datos
def enrich_data(collection_name='programas', create_backup=True):
    # Crear backup si se solicita
    if create_backup:
        backup_name = backup_collection(collection_name)
        if not backup_name:
            return False
    
    # Agrupar programas por universidad y ciudad
    pipeline = [
        {
            "$group": {
                "_id": {
                    "universidad": "$universidad",
                    "ciudad": "$ciudad"
                },
                "programas": {
                    "$push": {
                        "_id": "$_id",
                        "nombre": "$programa",
                        "lineas_investigacion": {
                            "$split": ["$linea_investigacion", "\n\n"]
                        },
                        "url": "$url",
                        "linea_investigacion_raw": "$linea_investigacion"
                    }
                }
            }
        }
    ]
    
    universidades = list(db[collection_name].aggregate(pipeline))
    
    # Variables para seguimiento
    total_universities = len(universidades)
    total_programs = sum(len(uni["programas"]) for uni in universidades)
    processed_universities = 0
    processed_programs = 0
    
    # Mostrar barra de progreso
    progress = widgets.FloatProgress(
        value=0,
        min=0,
        max=total_programs,
        description='Progreso:',
        bar_style='info',
        style={'bar_color': '#0abde3'},
        orientation='horizontal'
    )
    
    info_text = widgets.HTML(value=f"Procesando {total_universities} universidades con {total_programs} programas...")
    current_item = widgets.HTML(value="")
    
    display(widgets.VBox([info_text, progress, current_item]))
    
    # Procesar cada universidad
    for uni in universidades:
        universidad = uni["_id"]["universidad"]
        ciudad = uni["_id"]["ciudad"]
        processed_universities += 1
        
        current_item.value = f"<b>Universidad:</b> {universidad} ({processed_universities}/{total_universities})"
        
        # Obtener coordenadas para la ciudad
        coords = get_coordinates(ciudad)
        
        # Obtener métricas de ciudad
        city_metrics = get_city_metrics(ciudad)
        
        # Procesar cada programa
        for prog in uni["programas"]:
            program_id = prog["_id"]
            processed_programs += 1
            progress.value = processed_programs
            current_item.value = f"<b>Universidad:</b> {universidad} - <b>Programa:</b> {prog['nombre']} ({processed_programs}/{total_programs})"
            
            # Generar resumen si no existe
            if db[collection_name].find_one({"_id": program_id, "resumen": {"$exists": True}}) is None:
                resumen = generate_summary(prog["linea_investigacion_raw"])
                db[collection_name].update_one({"_id": program_id}, {"$set": {"resumen": resumen}})
            
            # Actualizar coordenadas y métricas de ciudad
            update_data = {}
            if coords:
                update_data["coords"] = coords
            if city_metrics:
                update_data["ciudad_metrics"] = city_metrics
            
            if update_data:
                db[collection_name].update_one({"_id": program_id}, {"$set": update_data})
        
        # Generar estadísticas para la universidad
        stats = generate_stats(universidad, uni["programas"])
        
        # Actualizar estadísticas en todos los programas de esta universidad
        db[collection_name].update_many(
            {"universidad": universidad},
            {"$set": {"stats": stats}}
        )
    
    success_msg(f"Enriquecimiento de datos completado: {processed_universities} universidades, {processed_programs} programas procesados.")
    return True

### Enriquecer datos

⚠️ **Nota**: Este proceso puede tardar varios minutos ya que realiza múltiples llamadas a la API de OpenAI.

In [ ]:
# Interfaz para enriquecer datos
create_backup_checkbox = widgets.Checkbox(
    value=True,
    description='Crear backup antes de enriquecer',
    style={'description_width': 'initial'}
)

enrich_button = widgets.Button(
    description='Iniciar Enriquecimiento',
    button_style='primary',
    tooltip='Iniciar proceso de enriquecimiento de datos'
)

output = widgets.Output()

def on_enrich_button_click(b):
    with output:
        clear_output()
        print("Iniciando proceso de enriquecimiento...")
        enrich_data(create_backup=create_backup_checkbox.value)

enrich_button.on_click(on_enrich_button_click)

display(widgets.VBox([
    create_backup_checkbox,
    enrich_button,
    output
]))

## Visualización de Datos

### Carga de Datos para Visualización

In [ ]:
# Función para cargar datos de universidades desde MongoDB
def load_university_data(collection_name='programas'):
    try:
        # Agrupar por universidad
        pipeline = [
            {
                "$group": {
                    "_id": {
                        "universidad": "$universidad",
                        "ciudad": "$ciudad"
                    },
                    "programas": {"$push": "$$ROOT"},
                    "coords": {"$first": "$coords"},
                    "stats": {"$first": "$stats"},
                    "ciudad_metrics": {"$first": "$ciudad_metrics"},
                    "count": {"$sum": 1}
                }
            },
            {
                "$project": {
                    "_id": 0,
                    "nombre": "$_id.universidad",
                    "ciudad": "$_id.ciudad",
                    "programas": 1,
                    "coords": 1,
                    "stats": 1,
                    "ciudad_metrics": 1,
                    "count": 1
                }
            }
        ]
        
        universidades = list(db[collection_name].aggregate(pipeline))
        return universidades
    except Exception as e:
        error_msg(f"Error al cargar datos de universidades: {str(e)}")
        return []

# Cargar datos
universidades = load_university_data()

# Mostrar estadísticas básicas
if universidades:
    total_universities = len(universidades)
    total_programs = sum(uni["count"] for uni in universidades)
    cities = set(uni["ciudad"] for uni in universidades)
    
    print(f"Datos cargados: {total_universities} universidades con {total_programs} programas en {len(cities)} ciudades.")
else:
    print("No se pudieron cargar datos de universidades.")

### Mapa Interactivo de Universidades

In [ ]:
# Crear mapa interactivo
def create_interactive_map(universidades):
    # Crear mapa centrado en España
    m = folium.Map(location=[40.4168, -3.7038], zoom_start=6, tiles='CartoDB positron')
    
    # Añadir clúster de marcadores
    marker_cluster = MarkerCluster().add_to(m)
    
    # Añadir marcadores para cada universidad
    for uni in universidades:
        # Obtener coordenadas
        coords = None
        if "coords" in uni and uni["coords"]:
            coords = [uni["coords"]["lat"], uni["coords"]["lon"]]
        
        # Si no hay coordenadas en los datos, usar la ubicación predefinida
        if not coords:
            continue
        
        # Crear popup con información de la universidad
        popup_html = f"""
        <div style="width: 300px; max-height: 300px; overflow-y: auto;">
            <h3 style="color: #3c40c6;">{uni['nombre']}</h3>
            <p><strong>Ciudad:</strong> {uni['ciudad']}</p>
            <p><strong>Programas:</strong> {uni['count']}</p>
        """
        
        # Añadir estadísticas si están disponibles
        if "stats" in uni and uni["stats"]:
            popup_html += f"""
            <h4>Estadísticas Académicas</h4>
            <ul>
                <li><strong>Innovación:</strong> {uni['stats'].get('innovacion', 'N/A')}/10</li>
                <li><strong>Interdisciplinariedad:</strong> {uni['stats'].get('interdisciplinariedad', 'N/A')}/10</li>
                <li><strong>Impacto:</strong> {uni['stats'].get('impacto', 'N/A')}/10</li>
                <li><strong>Internacional:</strong> {uni['stats'].get('internacional', 'N/A')}/10</li>
                <li><strong>Aplicabilidad:</strong> {uni['stats'].get('aplicabilidad', 'N/A')}/10</li>
            </ul>
            """
        
        # Añadir métricas de ciudad si están disponibles
        if "ciudad_metrics" in uni and uni["ciudad_metrics"]:
            popup_html += f"""
            <h4>Calidad de Vida</h4>
            <ul>
                <li><strong>Costo de Vida:</strong> {uni['ciudad_metrics'].get('costo_vida', 'N/A')}/100</li>
                <li><strong>Calidad Servicio Médico:</strong> {uni['ciudad_metrics'].get('calidad_servicio_medico', 'N/A')}/10</li>
                <li><strong>Calidad Transporte:</strong> {uni['ciudad_metrics'].get('calidad_transporte', 'N/A')}/10</li>
                <li><strong>Calidad del Aire:</strong> {uni['ciudad_metrics'].get('calidad_aire', 'N/A')}/10</li>
            </ul>
            """
        
        # Lista de programas
        popup_html += f"""
        <h4>Programas</h4>
        <ul>
        """
        
        for prog in uni["programas"][:5]:  # Mostrar solo los primeros 5 programas para no saturar
            popup_html += f"<li>{prog['programa']}</li>"
        
        if len(uni["programas"]) > 5:
            popup_html += f"<li>... y {len(uni['programas']) - 5} más</li>"
        
        popup_html += "</ul></div>"
        
        # Crear popup y añadir marcador
        popup = folium.Popup(popup_html, max_width=350)
        
        # Añadir marcador al clúster
        folium.Marker(
            location=coords,
            popup=popup,
            tooltip=f"{uni['nombre']} ({uni['ciudad']})",
            icon=folium.Icon(color='blue', icon='graduation-cap', prefix='fa')
        ).add_to(marker_cluster)
    
    return m

# Crear y mostrar mapa
if universidades:
    map_with_universities = create_interactive_map(universidades)
    display(map_with_universities)
else:
    error_msg("No hay datos de universidades para mostrar en el mapa.")

### Gráficos Radiales para Universidades

In [ ]:
# Función para crear gráfico radial de universidad
def plot_university_radar(universidad):
    if not universidad.get('stats'):
        return None, "No hay estadísticas disponibles para esta universidad."
    
    # Preparar datos para el gráfico radial académico
    stats = universidad['stats']
    categories = ['Innovación', 'Interdisciplinariedad', 'Impacto', 'Internacional', 'Aplicabilidad']
    values = [stats.get('innovacion', 0), stats.get('interdisciplinariedad', 0), 
              stats.get('impacto', 0), stats.get('internacional', 0), stats.get('aplicabilidad', 0)]
    
    # Cerrar el polígono repitiendo el primer valor
    categories = categories + [categories[0]]
    values = values + [values[0]]
    
    # Crear figura con dos subplots: uno para estadísticas académicas y otro para métricas de ciudad
    fig = plt.figure(figsize=(18, 8))
    
    # Subplot para estadísticas académicas
    ax1 = fig.add_subplot(121, polar=True)
    ax1.set_theta_offset(np.pi / 2)
    ax1.set_theta_direction(-1)
    ax1.set_rlabel_position(0)
    plt.yticks([2, 4, 6, 8, 10], ["2", "4", "6", "8", "10"], color="grey", size=8)
    plt.ylim(0, 10)
    
    # Añadir datos al gráfico académico
    ax1.plot(np.linspace(0, 2*np.pi, len(categories)), values, linewidth=2, linestyle='solid', label=universidad['nombre'])
    ax1.fill(np.linspace(0, 2*np.pi, len(categories)), values, alpha=0.25)
    ax1.set_thetagrids(np.degrees(np.linspace(0, 2*np.pi, len(categories)-1, endpoint=False)), categories[:-1])
    ax1.set_title(f"Métricas Académicas: {universidad['nombre']}\n", fontsize=14, pad=20)
    
    # Subplot para métricas de ciudad si están disponibles
    if universidad.get('ciudad_metrics'):
        ax2 = fig.add_subplot(122, polar=True)
        ax2.set_theta_offset(np.pi / 2)
        ax2.set_theta_direction(-1)
        ax2.set_rlabel_position(0)
        plt.yticks([2, 4, 6, 8, 10], ["2", "4", "6", "8", "10"], color="grey", size=8)
        plt.ylim(0, 10)
        
        # Preparar datos para el gráfico radial de ciudad
        city_metrics = universidad['ciudad_metrics']
        city_categories = ['Serv. Médico', 'Transporte', 'Calidad Aire', 'Costo Vida (inv)', 'Vivienda']
        
        # Normalizar métricas a escala 0-10
        costo_vida_norm = 10 - min(10, max(0, city_metrics.get('costo_vida', 70) / 10))
        city_values = [
            city_metrics.get('calidad_servicio_medico', 7),
            city_metrics.get('calidad_transporte', 7),
            city_metrics.get('calidad_aire', 7),
            costo_vida_norm,
            7  # Valor por defecto para vivienda
        ]
        
        # Cerrar el polígono
        city_categories = city_categories + [city_categories[0]]
        city_values = city_values + [city_values[0]]
        
        # Añadir datos al gráfico de ciudad
        ax2.plot(np.linspace(0, 2*np.pi, len(city_categories)), city_values, linewidth=2, linestyle='solid', color='#0abde3', label=universidad['ciudad'])
        ax2.fill(np.linspace(0, 2*np.pi, len(city_categories)), city_values, alpha=0.25, color='#0abde3')
        ax2.set_thetagrids(np.degrees(np.linspace(0, 2*np.pi, len(city_categories)-1, endpoint=False)), city_categories[:-1])
        ax2.set_title(f"Calidad de Vida: {universidad['ciudad']}\n", fontsize=14, pad=20)
    
    plt.tight_layout()
    plt.subplots_adjust(wspace=0.1)
    
    return fig, None

# Selector de universidad para gráfico radial
def display_university_selector():
    if not universidades:
        error_msg("No hay datos de universidades para mostrar.")
        return
    
    # Crear dropdown para seleccionar universidad
    university_options = [(f"{uni['nombre']} ({uni['ciudad']})", i) for i, uni in enumerate(universidades)]
    university_dropdown = widgets.Dropdown(
        options=university_options,
        description='Universidad:',
        style={'description_width': 'initial'}
    )
    
    # Crear área para mostrar el gráfico
    output = widgets.Output()
    
    # Función para actualizar el gráfico
    def on_university_change(change):
        with output:
            clear_output()
            if change['new'] is not None:
                universidad = universidades[change['new']]
                fig, error = plot_university_radar(universidad)
                if error:
                    error_msg(error)
                else:
                    plt.show()
    
    university_dropdown.observe(on_university_change, names='value')
    
    # Mostrar la interfaz
    display(widgets.VBox([
        university_dropdown,
        output
    ]))
    
    # Mostrar el gráfico para la primera universidad si está disponible
    if university_options:
        with output:
            universidad = universidades[university_options[0][1]]
            fig, error = plot_university_radar(universidad)
            if error:
                error_msg(error)
            else:
                plt.show()

# Mostrar selector de universidad y gráfico radial
display_university_selector()

### Programas por Estado

In [ ]:
# Función para obtener resumen de estado de programas
def get_program_status_summary(collection_name='programas'):
    try:
        # Agregación para contar programas por estado
        pipeline = [
            {
                "$group": {
                    "_id": {"$ifNull": ["$status", "pendiente"]},
                    "count": {"$sum": 1}
                }
            },
            {
                "$project": {
                    "_id": 0,
                    "status": "$_id",
                    "count": 1
                }
            },
            {"$sort": {"count": -1}}
        ]
        
        status_counts = list(db[collection_name].aggregate(pipeline))
        
        # Convertir a DataFrame para visualización
        status_df = pd.DataFrame(status_counts)
        if status_df.empty:
            return None
        
        return status_df
    except Exception as e:
        error_msg(f"Error al obtener resumen de estado de programas: {str(e)}")
        return None

# Obtener y mostrar resumen de estado
status_df = get_program_status_summary()

if status_df is not None:
    # Crear gráfico de barras
    plt.figure(figsize=(10, 6))
    bars = plt.bar(status_df['status'], status_df['count'])
    
    # Colorear barras según el estado
    for i, bar in enumerate(bars):
        status = status_df.iloc[i]['status']
        color = status_colors.get(status, '#aaaaaa')
        bar.set_color(color)
    
    plt.title('Programas por Estado', fontsize=16)
    plt.xlabel('Estado', fontsize=12)
    plt.ylabel('Número de Programas', fontsize=12)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.xticks(rotation=45)
    
    # Añadir valores sobre las barras
    for i, v in enumerate(status_df['count']):
        plt.text(i, v + 0.5, str(v), ha='center')
    
    plt.tight_layout()
    plt.show()
    
    # Mostrar tabla de datos
    display(status_df.set_index('status'))
else:
    error_msg("No se pudieron obtener datos de estado de programas.")

### Consultas y Filtros Personalizados

In [ ]:
# Función para buscar programas por palabra clave
def search_programs(keyword, collection_name='programas'):
    try:
        # Crear expresión regular para búsqueda insensible a mayúsculas/minúsculas
        regex = {"$regex": keyword, "$options": "i"}
        
        # Buscar en varios campos
        query = {
            "$or": [
                {"universidad": regex},
                {"ciudad": regex},
                {"programa": regex},
                {"linea_investigacion": regex},
                {"resumen": regex}
            ]
        }
        
        # Ejecutar consulta
        results = list(db[collection_name].find(query).limit(100))  # Limitar a 100 resultados
        
        return results
    except Exception as e:
        error_msg(f"Error al buscar programas: {str(e)}")
        return []

# Interfaz de búsqueda
search_input = widgets.Text(
    value='',
    placeholder='Ingrese palabra clave para buscar...',
    description='Buscar:',
    style={'description_width': 'initial'}
)

search_button = widgets.Button(
    description='Buscar',
    button_style='primary',
    tooltip='Buscar programas'
)

output = widgets.Output()

def on_search_button_click(b):
    with output:
        clear_output()
        keyword = search_input.value
        if not keyword:
            info_msg("Ingrese una palabra clave para buscar.")
            return
        
        print(f"Buscando programas con: '{keyword}'...")
        results = search_programs(keyword)
        
        if not results:
            info_msg(f"No se encontraron resultados para '{keyword}'.")
            return
        
        print(f"Se encontraron {len(results)} resultados.")
        print("\nResultados:")
        
        for i, prog in enumerate(results, 1):
            print(f"\n{i}. {prog.get('programa', 'Sin nombre')}")
            print(f"   Universidad: {prog.get('universidad', 'No especificada')}")
            print(f"   Ciudad: {prog.get('ciudad', 'No especificada')}")
            print(f"   Estado: {prog.get('status', 'pendiente')}")
            
            if 'resumen' in prog and prog['resumen']:
                print(f"   Resumen: {prog['resumen'][:150]}...")

search_button.on_click(on_search_button_click)

display(widgets.VBox([
    widgets.HBox([search_input, search_button]),
    output
]))

### Vista Detallada de un Programa

In [ ]:
# Función para obtener detalles de un programa
def get_program_details(program_id, collection_name='programas'):
    try:
        # Convertir string a ObjectId si es necesario
        if isinstance(program_id, str):
            program_id = ObjectId(program_id)
        
        # Buscar programa por ID
        programa = db[collection_name].find_one({"_id": program_id})
        if not programa:
            return None
        
        return programa
    except Exception as e:
        error_msg(f"Error al obtener detalles del programa: {str(e)}")
        return None

# Interfaz para visualizar un programa por ID
program_id_input = widgets.Text(
    value='',
    placeholder='Ingrese ID del programa...',
    description='ID Programa:',
    style={'description_width': 'initial'}
)

view_button = widgets.Button(
    description='Ver Detalles',
    button_style='info',
    tooltip='Ver detalles del programa'
)

output = widgets.Output()

def on_view_button_click(b):
    with output:
        clear_output()
        program_id = program_id_input.value
        if not program_id:
            info_msg("Ingrese un ID de programa válido.")
            return
        
        try:
            programa = get_program_details(program_id)
            
            if not programa:
                error_msg(f"No se encontró programa con ID: {program_id}")
                return
            
            # Mostrar detalles del programa
            print(f"\n📚 {programa.get('programa', 'Sin nombre')}")
            print(f"\n🏛️ Universidad: {programa.get('universidad', 'No especificada')}")
            print(f"📍 Ciudad: {programa.get('ciudad', 'No especificada')}")
            print(f"🔗 URL: {programa.get('url', 'No disponible')}")
            print(f"📊 Estado: {programa.get('status', 'pendiente')}")
            
            print("\n🔬 Líneas de Investigación:")
            lineas = programa.get('linea_investigacion', '').split('\n\n')
            for linea in lineas:
                if linea.strip():
                    print(f"• {linea.strip()}")
            
            if 'resumen' in programa and programa['resumen']:
                print(f"\n📝 Resumen: {programa['resumen']}")
            
            if 'stats' in programa and programa['stats']:
                print("\n📊 Estadísticas Académicas:")
                stats = programa['stats']
                print(f"• Innovación: {stats.get('innovacion', 'N/A')}/10")
                print(f"• Interdisciplinariedad: {stats.get('interdisciplinariedad', 'N/A')}/10")
                print(f"• Impacto: {stats.get('impacto', 'N/A')}/10")
                print(f"• Internacional: {stats.get('internacional', 'N/A')}/10")
                print(f"• Aplicabilidad: {stats.get('aplicabilidad', 'N/A')}/10")
            
            if 'ciudad_metrics' in programa and programa['ciudad_metrics']:
                print(f"\n🏙️ Métricas de {programa['ciudad']}:")
                metrics = programa['ciudad_metrics']
                print(f"• Costo de Vida: {metrics.get('costo_vida', 'N/A')}/100")
                print(f"• Calidad Servicio Médico: {metrics.get('calidad_servicio_medico', 'N/A')}/10")
                print(f"• Calidad Transporte: {metrics.get('calidad_transporte', 'N/A')}/10")
                print(f"• Calidad del Aire: {metrics.get('calidad_aire', 'N/A')}/10")
        except Exception as e:
            error_msg(f"Error al procesar programa: {str(e)}")

view_button.on_click(on_view_button_click)

display(widgets.VBox([
    widgets.HBox([program_id_input, view_button]),
    output
]))